<a href="https://colab.research.google.com/github/ihadanny/nlp_3523/blob/master/nlp_assignment_2_hadanny_dayan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

This assignment is about training and evaluating a POS tagger with some real data. The dataset is available through NLTK, a Python NLP package.

**Part 1** (no actions required)

The dataset is composed of a set of sentences. Each sentence is a list of tuples of a word and a tag, as provided by human annotators.
You should split the data to train and test sets in the following way:


In [2]:
import numpy as np
import operator
import nltk
from nltk.corpus import treebank 
nltk.download('treebank')
print(f"Number of sentences: {len(treebank.tagged_sents())}")

train_data = treebank.tagged_sents()[:3000] 
test_data = treebank.tagged_sents()[3000:] 
print(train_data[0])

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
Number of sentences: 3914
[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]


**Part 2**

Write a class simple_tagger, with methods *train* and *evaluate*. The method *train* receives the data as a list of treebank sentences, as presented below, and use it for training the tagger. In this case, it should learn a simple map of words to tags, defined as the most frequent tag for every word (in case there is more than one tag, select one randomly). The map should be stored as a class member for evaluation.

The method evaluate receives the data as a list of treebanl sentences, as presented above, and use it to evaluate the tagger performance. Specifically, it should calculate the word and sentence level accuracy.
The evaluation process is simply going word by word, querying the map (created by the train method) for each word’s tag and compare it to the true tag of that word. The word-level accuracy is the number of successes divided by the number of words. For OOV (out of vocabulary, or unknown) words, the tagger should assign the most frequent tag in the entire training set. The function should return the two numbers: word level accuracy and sentence level accuracy.


In [0]:
import collections
class simple_tagger:
  def train(self, data):
    occs = [x for sent in data for x in sent]
    occs = collections.Counter(occs)
    freq = collections.defaultdict(list)
    for (word, tag), cnt in occs.items():
      freq[word].append((cnt, tag))
    self.most_freq_tag_per_word = {word: max(pos_list)[1] for word, pos_list in freq.items()}
    all_tags = [tag for sent in data for word, tag in sent]
    all_tags = collections.Counter(all_tags)
    self.most_freq_tag_overall = all_tags.most_common(1)[0][0]
    print(self.most_freq_tag_overall)

  def evaluate(self, data):
    word_successes, sent_successes = 0, 0
    for sent in data:
      entire_sent_success = True
      for word, true_tag in sent:
        tag = self.most_freq_tag_per_word[word] if word in self.most_freq_tag_per_word else self.most_freq_tag_overall
        if tag == true_tag:
          word_successes += 1
        else: 
          entire_sent_success = False
      if entire_sent_success:
        sent_successes += 1
    words, sents = len([word for sent in data for word in sent]), len(data)
    return word_successes / words, sent_successes / sents


In [58]:
tagger = simple_tagger()
tagger.train(train_data)
simple_res = tagger.evaluate(test_data)
print(simple_res)


NN
(0.875415497517807, 0.08533916849015317)


**Part 3**

Similar to part 2, write the class hmm_tagger, which implements HMM tagging. The method *train* should build the matrices A, B and Pi, from the data as discussed in class. The method *evaluate* should find the best tag sequence for every input sentence, using he Viterbi decoding algorithm, and then calculate the word and sentence level accuracy using the gold-standard tags. I implemented the Viterbi algorithm for you in the next block, so you can should either plug it into your code or write your own Viterbi version.

Additional guidance:
1. The matrix B represents the probabilities of seeing a word within each POS label.
Since B is a matrix, you should build a dictionary that maps every unique word in the corpus to a serial numeric id (starting with 0). This way, every column in B represents the word that it’s id matches the index of the column.
2. During evaluation, you should first convert each word into it’s index and create the observation array to be given to Viterbi, as a list of ids. OOV words should be assigned with a random tag. To make sure Viterbi works appropriately, you can simply break the sentence into multiple segments every time you see OOV word, and decode every segment individually by Viterbi.


In [0]:
# Viterbi
def viterbi (word_list, A, B, Pi):

    # initialization
    T = len(word_list)
    N = A.shape[0] # number of tags

    delta_table = np.zeros((N, T)) # initialise delta table
    psi = np.zeros((N, T))  # initialise the best path table

    delta_table[:,0] = B[:, word_list[0]] * Pi

    for t in range(1, T):
        for s in range (0, N):
            trans_p = delta_table[:, t-1] * A[:, s]
            psi[s, t], delta_table[s, t] = max(enumerate(trans_p), key=operator.itemgetter(1))
            delta_table[s, t] = delta_table[s, t] * B[s, word_list[t]]

    # Back tracking
    seq = np.zeros(T);
    seq[T-1] =  delta_table[:, T-1].argmax()
    for t in range(T-1, 0, -1):
      #print(seq[t])
      seq[t-1] = psi[int(seq[t]),t]

    return seq

# A simple example to run the algorithm:

# A = np.array([[0.3, 0.7], [0.2, 0.8]])
# B = np.array([[0.1, 0.1, 0.3, 0.5], [0.3, 0.3, 0.2, 0.2]])
# Pi = np.array([0.4, 0.6])
# print(viterbi([3, 3, 3, 3], A, B, Pi))

In [0]:
import random
class hmm_tagger:
  def sent_to_indexes(self, sent):
    res = []
    for word, tag in sent:
      tag = self.tags[tag]
      # replacing oov with a word from the most common tag, to avoid unfair advantage for simple
      word = self.words[word] if word in self.words else self.word_for_most_freq_tag
      res.append((word, tag))
    return res

  def train(self, data):
    self.words = set([word for sent in data for word, tag in sent])
    self.words = {word: idx for idx, word in enumerate(self.words)}
    self.tags = set([tag for sent in data for word, tag in sent])
    self.tags = {tag: idx for idx, tag in enumerate(self.tags)}

    all_tags = [tag for sent in data for word, tag in sent]
    all_tags = collections.Counter(all_tags)
    most_freq_tag_overall = all_tags.most_common(1)[0][0]
    print(most_freq_tag_overall)
    self.word_for_most_freq_tag = max([word for sent in data for word, tag in sent if tag == most_freq_tag_overall])
    print(self.word_for_most_freq_tag)
    self.word_for_most_freq_tag = self.words[self.word_for_most_freq_tag]

    N, T = len(self.tags), len(self.words)
    self.A, self.B, self.Pi = np.zeros((N, N)), np.zeros((N, T)), np.zeros(N)

    for sent in data:
      sent = self.sent_to_indexes(sent)
      for idx, (word,tag) in enumerate(sent):
        self.B[tag, word] += 1
        if idx == 0:
          self.Pi[tag] += 1
        else:
          _, prev_tag = sent[idx-1]
          self.A[prev_tag, tag] += 1

    def norm_by_row(mat):
      return (mat.T / mat.T.sum(axis = 0)).T
    self.A, self.B, self.Pi = norm_by_row(self.A), norm_by_row(self.B), norm_by_row(self.Pi) 

  def evaluate(self, data):
    N, T = len(self.tags), len(self.words)

    word_successes, sent_successes = 0, 0
    for sent in data:
      sent = self.sent_to_indexes(sent)
      words = [word for word, tag in sent]
      true_tags =[tag for word, tag in sent]
      tags = viterbi(words, self.A, self.B, self.Pi)

      entire_sent_success = True
      for idx in range(len(tags)):
        if tags[idx] == true_tags[idx]:
          word_successes += 1
        else: 
          entire_sent_success = False
      if entire_sent_success:
        sent_successes += 1
    words, sents = len([word for sent in data for word in sent]), len(data)
    return word_successes / words, sent_successes / sents




In [59]:

tagger = hmm_tagger()
tagger.train(train_data)
hmm_res = tagger.evaluate(test_data)
print(hmm_res)


NN
zone
(0.7975393913231168, 0.11050328227571116)


**Part 4**

Compare the results obtained from both taggers and a MEMM tagger, implemented by NLTK, over the test data. To train the NLTK MEMM tagger you should execute the following lines (it may take some time to train...):

In [8]:
from nltk.tag import tnt 

tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(train_data)
print(tnt_pos_tagger.evaluate(test_data))

0.875545003237643


In [60]:

def evaluate(data):
  word_successes, sent_successes = 0, 0
  tnt_tagger_preds = tnt_pos_tagger.tag_sents([[word for word,_ in test_sent] for test_sent in data])
  tnt_tagger_pred_labels = [[tag for _, tag in sentence] for sentence in tnt_tagger_preds]
  for idx, sent in enumerate(data):
    true_tags = [tag for word, tag in sent]
    tags = tnt_tagger_pred_labels[idx]
    entire_sent_success = True
    for idx in range(len(tags)):
      if tags[idx] == true_tags[idx]:
        word_successes += 1
      else: 
        entire_sent_success = False
    if entire_sent_success:
      sent_successes += 1
  words, sents = len([word for sent in data for word in sent]), len(data)
  return word_successes / words, sent_successes / sents

memm_res = evaluate(test_data)
print(memm_res)

(0.875545003237643, 0.09846827133479212)


TODO: Print both, word level and sentence level accuracy for all the three taggers in a table.

In [63]:
import pandas as pd
pd.DataFrame(columns = ['model', 'word_level_acc', 'sent_level_acc'],
             data=[('simple', simple_res[0], simple_res[1]),
              ('hmm', hmm_res[0], hmm_res[1]),
              ('memm', memm_res[0], memm_res[1]),
              ])

,model,word_level_acc,sent_level_acc
0,simple,0.875415,0.085339
1,hmm,0.797539,0.110503
2,memm,0.875545,0.098468
